# INTRO
Code corresponding to my [KDnuggets article]()

In [8]:
! pip install pandasql


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [6]:
import pandas as pd

# URL of the CSV file
url = "https://gist.githubusercontent.com/armgilles/194bcff35001e7eb53a2a8b441e8b2c6/raw/92200bc0a673d5ce2110aaad4544ed6c4010f687/pokemon.csv"

# Read the CSV file directly from the URL
data = pd.read_csv(url)

# Display the first few rows of the dataset
data.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


# COMPARING A SIMPLE QUERY IN BOTH SQL AND PANDAS

In [12]:
from pandasql import sqldf
import pandas as pd

sqldf('''
      SELECT "#", Name, Total
      FROM data 
      ORDER  BY Total
      LIMIT 5''')

,#,Name,Total
0,191,Sunkern,180
1,298,Azurill,190
2,401,Kricketot,194
3,10,Caterpie,195
4,13,Weedle,195


In [16]:
# Sort the DataFrame by the "Total" column in descending order and display the top 5
data[["#", "Name", "Total"]].sort_values(by="Total", ascending=True).head(5)


,#,Name,Total
206,191,Sunkern,180
322,298,Azurill,190
446,401,Kricketot,194
288,265,Wurmple,195
16,13,Weedle,195


In [19]:
data

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True


# WORKING WITH PYSQLDF

## GENERATE MULTIPLE TABLES

In [28]:
# SQL
types = sqldf('''
      SELECT "#", Name, "Type 1", "Type 2"
      FROM data''')

legendaries = sqldf('''
      SELECT "#", Name, Legendary
      FROM data''')

generations = sqldf('''
      SELECT "#", Name, Generation
      FROM data''')

features = sqldf('''
      SELECT "#", Name, Total, HP, Attack, Defense, "Sp. Atk", "Sp. Def","Speed"
      FROM data''')

In [29]:
# PYTHON
# Selecting columns for types
types = data[['#', 'Name', 'Type 1', 'Type 2']]

# Selecting columns for legendaries
legendaries = data[['#','Name', 'Legendary']]

# Selecting columns for generations
generations = data[['#','Name', 'Generation']]

# Selecting columns for features
features = data[['#','Name', 'Total', 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']]

## JOINs

In [31]:
types_features = sqldf('''
      SELECT 
        t1.*, 
        t2.Total, 
        t2.HP, 
        t2.Attack,
        t2.Defense,
        t2."Sp. Atk",
        t2."Sp. Def",
        t2."Speed"
      FROM types AS t1
      LEFT JOIN features AS t2
        ON  t1."#" = t2."#"
        AND t1.Name = t2.Name
''')

types_features

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80
4,4,Charmander,Fire,None,309,39,52,43,60,50,65
...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80


In [32]:
# Performing a left join between `types` and `features` on the columns "#" and "Name"
types_features = types.merge(
    features,
    on=['#', 'Name'],
    how='left'
)

types_features


,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65
...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80


## Get the top 5 pokemons with the highest defense

In [36]:
top_5_defense = sqldf('''
      SELECT 
        Name, Defense
      FROM features
      ORDER BY Defense DESC
      LIMIT 5
''')

top_5_defense

,Name,Defense
0,SteelixMega Steelix,230
1,Shuckle,230
2,AggronMega Aggron,230
3,Steelix,200
4,Regirock,200


In [37]:
# Selecting the Name and Defense columns from `features`, sorting by Defense in descending order, and limiting to the top 5
top_5_defense = features[['Name', 'Defense']].sort_values(by='Defense', ascending=False).head(5)

top_5_defense


,Name,Defense
230,Shuckle,230
224,SteelixMega Steelix,230
333,AggronMega Aggron,230
223,Steelix,200
414,Regirock,200
